### **환경 설정**

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.8 MB/s eta 0:11:03tcmalloc: large alloc 1147494400 bytes == 0x39e86000 @  0x7f3187634615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:11:46tcmalloc: large alloc 1434370048 bytes == 0x7e4dc000 @  0x7f3187634615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.3 MB/s eta 0:08:55tcmalloc: large alloc 1792966656 bytes =

In [3]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.24.1
11.1
GCC 7.3


In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### **데이터셋 다운로드**

In [6]:
# Download dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="MY_API_KEY")
project = rf.workspace("sketch").project("sketch-detection")
dataset = project.version(2).download("coco")

     |████████████████████████████████| 145 kB 11.6 MB/s 
     |████████████████████████████████| 178 kB 50.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.8 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 138 kB 51.3 MB/s 
     |████████████████████████████████| 596 kB 43.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=fa355790bb8ef9b2dc66aed2b085ddb3e5ab9fe51cc9a6ae16f4c1332fe4cbc4
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=490ecf9d394cebb7deb91d875ba9abaaae6c19fac4264a2e0c992266e5ce220f
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wge

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to sketch-detection-2 in coco:: 100%|██████████| 3309/3309 [00:04<00:00, 805.12it/s]


### **Faster R-CNN**

<img src="https://user-images.githubusercontent.com/40661020/143881188-ab87720f-5059-4b4e-a928-b540fb8fb84d.png" width=500/>

In [10]:
from mmcv import Config
from mmdet.apis import set_random_seed

cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntrop

In [11]:
# config 설정

data_root = '/content/mmdetection/sketch-detection-2/'
ann_file = '_annotations.coco.json'

cfg.data_root = data_root

cfg.data.train.ann_file = data_root + 'train/' + ann_file
cfg.data.train.img_prefix = data_root + 'train/'
cfg.data.train.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.val.ann_file = data_root + 'valid/' + ann_file
cfg.data.val.img_prefix = data_root + 'valid/'
cfg.data.val.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.data.test.ann_file = data_root + 'test/' + ann_file
cfg.data.test.img_prefix = data_root + 'test/'
cfg.data.test.classes = (
                      'button', 'checkbox_checked', 'checkbox_unchecked',
                      'chip', 'data_table', 'dropdown_menu',
                      'floating_action_button', 'image', 'radiobutton_checked',
                      'radiobutton_unchecked', 'slider', 'switch_disabled',
                      'switch_enabled', 'text_field', 'text_area')

cfg.optimizer.lr = 0.0025

cfg.model.roi_head.bbox_head.num_classes = 15

cfg.runner.max_epochs = 30
cfg.max_epochs = 30

cfg.work_dir = 'working/faster_rcnn'

cfg.seed = 0

cfg.gpu_ids = [0]
cfg.device = 'cuda'

cfg.log_config.interval = 100

In [12]:
print(cfg.pretty_text)

# config 파일 저장
with open("faster_rcnn.py","w") as f:
  f.write(cfg.pretty_text)

!cp faster_rcnn.py /content/drive/MyDrive/model/faster_rcnn

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntrop

In [ ]:
# 학습

import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Save checkpoints
!cp -r /content/mmdetection/working/faster_rcnn /content/drive/MyDrive/model

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-05-14 05:46:41,494 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-05-14 05:46:41,506 - mmdet - INFO - Start running, host: root@2819bf16a767, work_dir: /content/mmdetection/working/yolof
2022-05-14 05:46:41,507 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook     

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-05-14 05:47:07,395 - mmdet - INFO - Epoch [1][100/1352]	lr: 4.970e-04, eta: 2:53:47, time: 0.258, data_time: 0.030, memory: 4029, loss_rpn_cls: 0.0789, loss_rpn_bbox: 0.1132, loss_cls: 0.2097, acc: 94.3018, loss_bbox: 0.1524, loss: 0.5542
2022-05-14 05:47:31,167 - mmdet - INFO - Epoch [1][200/1352]	lr: 9.965e-04, eta: 2:46:37, time: 0.238, data_time: 0.009, memory: 4029, loss_rpn_cls: 0.0294, loss_rpn_bbox: 0.0438, loss_cls: 0.1300, acc: 96.5547, loss_bbox: 0.1322, loss: 0.3354
2022-05-14 05:47:54,699 - mmdet - INFO - Epoch [1][300/1352]	lr: 1.496e-03, eta: 2:43:26, time: 0.235, data_time: 0.008, memory: 4029, loss_rpn_cls: 0.0283, loss_rpn_bbox: 0.0306, loss_cls: 0.1023, acc: 97.2305, loss_bbox: 0.1053, loss: 0.2665
2022-05-14 05:48:18,912 - mmdet - INFO - Epoch [1][400/1352]	lr: 1.996e-03, eta: 2:42:46, time: 0.242, data_time: 0.010, memory: 4029, loss_rpn_cls: 0.0245, loss_rpn_bbox: 0.0256, loss_cls: 0.0954, acc: 97.3564, loss_bbox: 0.0959, loss: 0.2414
2022-05-14 05:48:42,616 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 14.4 task/s, elapsed: 21s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 298/298, 13.9 task/s, elapsed: 21s, ETA:     0s

2022-05-14 08:42:05,490 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-05-14 08:42:06,141 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.840
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.892
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.890
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.840
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.951
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.951
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.951
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=

DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.14s).
cp: cannot stat '/content/mmdetection/working/faster_rcnn': No such file or directory
